In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

    
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [43]:
from typing import Any, Dict


examples = [
    {
        "country": "France",
        "answer" : """
        Here is what I know:
        Capital: Paris
        Language: France
        Food: Wine and Cheese
        Currency: Euro
        """ 
    },
    {
        "country": "Italy",
        "answer" : """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [45]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "What do you know about {country}?"),
#     ("ai", "{answer}")
# # ])
# prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples
# )
# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert, you give short answers."),
#     example_prompt,
#     ("human", "What do you know about {country}?")
# ])

# chain = final_prompt | chat
prompt.format(country="Germany")

# chain = prompt | chat
# chain.invoke({
#     "country":"Germany"
# })


'Human: Italy\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Germany?'

In [47]:
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")
prompt.format(country="Germany")

'What is the capital of Germany'

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer = "Arrrrg! That is a secret",
    question="What is your favorite food?"
)

chain = full_prompt | chat
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1
)
chat.predict("How do you make italian pasta?")

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    a = chat.predict("What is the receipe for soju")
    b = chat.predict("What is the receipe for bread")
    print(a, b, "\n")
    print(usage)